In [4]:
# Standard libraries
import os
import sys
import json
from dotenv import load_dotenv

# Third-party libraries
import pandas as pd

# Add project root to sys.path
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

# Local modules
from src.classifiers import is_uae_real_estate_query, llm_classifier
from src.tools import extract_data_intent, safe_dataframe_tool, create_plotly_code,get_openai_llm
from src.geo_tools import generate_google_maps_html

load_dotenv()

True

In [12]:
query = extract_data_intent.invoke("show histogram for the properties prices in this uae database and set the number of bins to 15")

In [23]:
df = pd.DataFrame(json.loads(result)["result"])

In [20]:
data = pd.read_parquet("/home/zcemg08/projects/chatbot/datasets/my-org/clean2/data.parquet")

In [22]:
data[(data["City"] == "Dubai") &  (data["addedOn_year_month"] == "2024-03") & (data["bedrooms"] == 1)]

,title,bathrooms,bedrooms,type,price,verified,priceDuration,sizeMin,furnishing,description,latitude,longitude,City,addedOn_year_month,addedOn_month_name,addedOn_month_num,addedOn_day_num,Area
316,Fully Furnished | Large Terrace | High ROI,1,1,Residential for Sale,830000,True,sell,32,YES,Lacon International is delighted to present th...,25.004149,55.286234,Dubai,2024-03,March,3,16,Town Square
394,FIVE Specialist | 10% ROI | Residential Resell,1,1,Residential for Sale,5200000,True,sell,32,YES,Residential Tower One Bedroom RESELL. Project ...,25.080890,55.136774,Dubai,2024-03,March,3,27,Jumeirah Beach Residence
520,Desirably Located | Dubai Marina | Modern buil...,2,1,Residential for Sale,1899000,True,sell,32,NO,Le-Pax Real Estate delighted to present to you...,25.068635,55.131475,Dubai,2024-03,March,3,21,Dubai Marina


In [35]:
data["Area"].unique()

array(['Business Bay', 'Jumeirah Beach Residence', 'Al Furjan', 'Meadows',
       'Dubai Marina', 'Villa', 'Springs', 'Dubai Silicon Oasis',
       'Dubai Land', 'Jumeirah Village', 'Hayyan', 'Jebel Ali',
       'Al Reem Island', 'Damac Hills', 'Arabian Ranches', 'Al Barari',
       'Khor Fakkan', 'Palm Jumeirah', 'City Walk', 'Downtown Dubai',
       'Culture Village', 'Dubai Sports City', 'Muwaileh',
       'Dubai Hills Estate', 'Jumeirah', 'Lakes',
       'Dubai Residence Complex', 'Al Jubail Island', 'Manama',
       'Al Riqqa', 'Al Ghadeer', 'Discovery Gardens', 'Al Shamkha',
       'Zabeel', 'Views', 'Difc', 'Al Nahda', 'Town Square', 'Meydan',
       'Motor City', 'Dubai Festival City', 'Majan', 'Nad Al Sheba',
       'Al Jaddaf', 'International City', 'Al Marjan Island', 'Al Warsan',
       'Wadi Al Safa', 'Barsha Heights', 'Al Wasl', 'Al Mamzar',
       'Wasl Gate', 'Dubai Industrial City', 'Al Dhait', 'Al Qurm',
       'Yas Island', 'Dubai Harbour', 'Al Tallah', 'Saadiyat Isl

In [37]:
data_cities = pd.DataFrame([{'City': 'Dubai', 'price': 2500000}, {'City': 'Dubai', 'price': 7200000}, {'City': 'Dubai', 'price': 950000}, {'City': 'Dubai', 'price': 9999990}, {'City': 'Dubai', 'price': 2100000}, {'City': 'Dubai', 'price': 6000000}, {'City': 'Dubai', 'price': 5699000}, {'City': 'Dubai', 'price': 1790000}, {'City': 'Dubai', 'price': 3150000}, {'City': 'Dubai', 'price': 3150000}, {'City': 'Dubai', 'price': 2200000}, {'City': 'Dubai', 'price': 1350000}, {'City': 'Dubai', 'price': 3400000}, {'City': 'Sharjah', 'price': 2089000}, {'City': 'Dubai', 'price': 2593860}, {'City': 'Dubai', 'price': 4100000}, {'City': 'Dubai', 'price': 4000000}, {'City': 'Dubai', 'price': 6100000}, {'City': 'Dubai', 'price': 15500000}, {'City': 'Dubai', 'price': 19750000}, {'City': 'Dubai', 'price': 13500000}, {'City': 'Dubai', 'price': 3000000}, {'City': 'Dubai', 'price': 2550000}, {'City': 'Dubai', 'price': 3350000}, {'City': 'Dubai', 'price': 9000000}, {'City': 'Sharjah', 'price': 2399000}, {'City': 'Dubai', 'price': 5800000}, {'City': 'Dubai', 'price': 18500000}, {'City': 'Fujairah', 'price': 3099000}, {'City': 'Fujairah', 'price': 2099000}, {'City': 'Dubai', 'price': 8500000}, {'City': 'Sharjah', 'price': 1990000}, {'City': 'Dubai', 'price': 34675000}, {'City': 'Dubai', 'price': 6000000}, {'City': 'Dubai', 'price': 8600000}, {'City': 'Dubai', 'price': 4500000}, {'City': 'Dubai', 'price': 2900000}, {'City': 'Dubai', 'price': 4800000}, {'City': 'Dubai', 'price': 3550000}, {'City': 'Dubai', 'price': 6000000}, {'City': 'Dubai', 'price': 12250000}, {'City': 'Dubai', 'price': 4900000}, {'City': 'Dubai', 'price': 16000000}, {'City': 'Dubai', 'price': 3700000}, {'City': 'Dubai', 'price': 14200000}, {'City': 'Dubai', 'price': 3200000}, {'City': 'Dubai', 'price': 11500000}, {'City': 'Dubai', 'price': 7700000}, {'City': 'Dubai', 'price': 4895000}, {'City': 'Dubai', 'price': 15500000}, {'City': 'Dubai', 'price': 4500000}, {'City': 'Dubai', 'price': 3800000}, {'City': 'Dubai', 'price': 5150000}, {'City': 'Dubai', 'price': 7900000}, {'City': 'Dubai', 'price': 5700000}, {'City': 'Dubai', 'price': 50000000}, {'City': 'Dubai', 'price': 8750000}, {'City': 'Dubai', 'price': 8500000}, {'City': 'Dubai', 'price': 1380000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Dubai', 'price': 3600000}, {'City': 'Dubai', 'price': 700000}, {'City': 'Sharjah', 'price': 480000}, {'City': 'Dubai', 'price': 7800000}, {'City': 'Dubai', 'price': 52000000}, {'City': 'Dubai', 'price': 5700000}, {'City': 'Dubai', 'price': 2550000}, {'City': 'Dubai', 'price': 3200000}, {'City': 'Dubai', 'price': 11500000}, {'City': 'Dubai', 'price': 15900000}, {'City': 'Dubai', 'price': 2494320}, {'City': 'Dubai', 'price': 5190000}, {'City': 'Dubai', 'price': 35000000}, {'City': 'Dubai', 'price': 3700000}, {'City': 'Dubai', 'price': 3600000}, {'City': 'Dubai', 'price': 3350000}, {'City': 'Dubai', 'price': 9999999}, {'City': 'Dubai', 'price': 670000}, {'City': 'Dubai', 'price': 2450000}, {'City': 'Dubai', 'price': 5100000}, {'City': 'Abu Dhabi', 'price': 12500000}, {'City': 'Ajman', 'price': 125000}, {'City': 'Dubai', 'price': 2700000}, {'City': 'Sharjah', 'price': 950000}, {'City': 'Abu Dhabi', 'price': 2100000}, {'City': 'Abu Dhabi', 'price': 815000}, {'City': 'Dubai', 'price': 680000}, {'City': 'Dubai', 'price': 900000}, {'City': 'Abu Dhabi', 'price': 1650000}, {'City': 'Sharjah', 'price': 390000}, {'City': 'Dubai', 'price': 2200000}, {'City': 'Dubai', 'price': 5500000}, {'City': 'Dubai', 'price': 1900000}, {'City': 'Dubai', 'price': 2640000}, {'City': 'Sharjah', 'price': 5000000}, {'City': 'Dubai', 'price': 1900000}, {'City': 'Dubai', 'price': 8049999}, {'City': 'Dubai', 'price': 8000000}, {'City': 'Sharjah', 'price': 450000}, {'City': 'Dubai', 'price': 2700000}, {'City': 'Dubai', 'price': 4750000}, {'City': 'Dubai', 'price': 3600000}, {'City': 'Abu Dhabi', 'price': 7000000}, {'City': 'Dubai', 'price': 750000}, {'City': 'Dubai', 'price': 2170000}, {'City': 'Dubai', 'price': 1780000}, {'City': 'Dubai', 'price': 1500000}, {'City': 'Dubai', 'price': 2650000}, {'City': 'Dubai', 'price': 3300000}, {'City': 'Dubai', 'price': 1750000}, {'City': 'Dubai', 'price': 1950000}, {'City': 'Dubai', 'price': 3900000}, {'City': 'Dubai', 'price': 3800000}, {'City': 'Dubai', 'price': 7850000}, {'City': 'Sharjah', 'price': 4849000}, {'City': 'Dubai', 'price': 5000000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Dubai', 'price': 2690000}, {'City': 'Dubai', 'price': 5250000}, {'City': 'Dubai', 'price': 1080000}, {'City': 'Dubai', 'price': 2500000}, {'City': 'Dubai', 'price': 2600000}, {'City': 'Dubai', 'price': 1475000}, {'City': 'Dubai', 'price': 7000000}, {'City': 'Dubai', 'price': 1725000}, {'City': 'Dubai', 'price': 6500000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Dubai', 'price': 2600000}, {'City': 'Dubai', 'price': 1400000}, {'City': 'Dubai', 'price': 9000000}, {'City': 'Dubai', 'price': 8650000}, {'City': 'Dubai', 'price': 3050000}, {'City': 'Dubai', 'price': 2000000}, {'City': 'Dubai', 'price': 3600000}, {'City': 'Dubai', 'price': 31000000}, {'City': 'Dubai', 'price': 1025000}, {'City': 'Dubai', 'price': 22000000}, {'City': 'Dubai', 'price': 850000}, {'City': 'Dubai', 'price': 8450000}, {'City': 'Dubai', 'price': 2500000}, {'City': 'Dubai', 'price': 925000}, {'City': 'Dubai', 'price': 2650000}, {'City': 'Dubai', 'price': 2400000}, {'City': 'Dubai', 'price': 1675000}, {'City': 'Dubai', 'price': 2400000}, {'City': 'Dubai', 'price': 6300000}, {'City': 'Abu Dhabi', 'price': 2100000}, {'City': 'Dubai', 'price': 2700000}, {'City': 'Dubai', 'price': 1150000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Dubai', 'price': 3950000}, {'City': 'Dubai', 'price': 14980000}, {'City': 'Dubai', 'price': 4500000}, {'City': 'Dubai', 'price': 975000}, {'City': 'Dubai', 'price': 933000}, {'City': 'Dubai', 'price': 6300000}, {'City': 'Dubai', 'price': 1550000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Dubai', 'price': 5000000}, {'City': 'Dubai', 'price': 2250000}, {'City': 'Dubai', 'price': 2290000}, {'City': 'Dubai', 'price': 2780000}, {'City': 'Dubai', 'price': 2073058}, {'City': 'Dubai', 'price': 2800000}, {'City': 'Dubai', 'price': 6200000}, {'City': 'Dubai', 'price': 7850000}, {'City': 'Abu Dhabi', 'price': 6500000}, {'City': 'Dubai', 'price': 650000}, {'City': 'Dubai', 'price': 1650000}, {'City': 'Dubai', 'price': 3800000}, {'City': 'Dubai', 'price': 7600000}, {'City': 'Sharjah', 'price': 2049000}, {'City': 'Abu Dhabi', 'price': 14000000}, {'City': 'Abu Dhabi', 'price': 13600000}, {'City': 'Dubai', 'price': 3450000}, {'City': 'Dubai', 'price': 7599000}, {'City': 'Dubai', 'price': 1750000}, {'City': 'Dubai', 'price': 5100000}, {'City': 'Dubai', 'price': 5400000}, {'City': 'Dubai', 'price': 2100000}, {'City': 'Dubai', 'price': 6200000}, {'City': 'Dubai', 'price': 7000000}, {'City': 'Dubai', 'price': 8690000}, {'City': 'Dubai', 'price': 2250000}, {'City': 'Dubai', 'price': 2099999}, {'City': 'Dubai', 'price': 9500000}, {'City': 'Dubai', 'price': 1400000}, {'City': 'Dubai', 'price': 1300000}, {'City': 'Dubai', 'price': 1199000}, {'City': 'Dubai', 'price': 13000000}, {'City': 'Dubai', 'price': 4100000}, {'City': 'Dubai', 'price': 900000}, {'City': 'Dubai', 'price': 1800000}, {'City': 'Dubai', 'price': 11500000}, {'City': 'Dubai', 'price': 510000}, {'City': 'Dubai', 'price': 8650000}, {'City': 'Dubai', 'price': 4321000}, {'City': 'Dubai', 'price': 2450000}, {'City': 'Dubai', 'price': 3500000}, {'City': 'Dubai', 'price': 1580000}, {'City': 'Dubai', 'price': 1675000}, {'City': 'Dubai', 'price': 4399999}, {'City': 'Dubai', 'price': 2650000}, {'City': 'Dubai', 'price': 680000}, {'City': 'Dubai', 'price': 2550000}, {'City': 'Dubai', 'price': 56500000}, {'City': 'Dubai', 'price': 2400000}, {'City': 'Dubai', 'price': 3300000}, {'City': 'Dubai', 'price': 2400000}, {'City': 'Sharjah', 'price': 749000}, {'City': 'Dubai', 'price': 1849999}, {'City': 'Dubai', 'price': 2000000}, {'City': 'Dubai', 'price': 5650000}, {'City': 'Dubai', 'price': 8000000}, {'City': 'Dubai', 'price': 5250000}, {'City': 'Dubai', 'price': 1600000}, {'City': 'Dubai', 'price': 3950000}, {'City': 'Dubai', 'price': 3695000}, {'City': 'Dubai', 'price': 3100000}, {'City': 'Dubai', 'price': 11250000}, {'City': 'Dubai', 'price': 12500000}, {'City': 'Dubai', 'price': 3050000}, {'City': 'Dubai', 'price': 3750000}, {'City': 'Dubai', 'price': 14700000}, {'City': 'Dubai', 'price': 4400000}, {'City': 'Abu Dhabi', 'price': 2100000}, {'City': 'Dubai', 'price': 4000000}, {'City': 'Dubai', 'price': 2700000}, {'City': 'Dubai', 'price': 2650000}, {'City': 'Dubai', 'price': 910000}, {'City': 'Ajman', 'price': 200000}, {'City': 'Dubai', 'price': 560000}, {'City': 'Dubai', 'price': 760000}, {'City': 'Dubai', 'price': 3890000}, {'City': 'Dubai', 'price': 4000000}, {'City': 'Sharjah', 'price': 2387000}, {'City': 'Dubai', 'price': 780000}, {'City': 'Dubai', 'price': 2699999}, {'City': 'Dubai', 'price': 1899000}, {'City': 'Dubai', 'price': 850000}, {'City': 'Dubai', 'price': 4300000}, {'City': 'Abu Dhabi', 'price': 7150000}, {'City': 'Dubai', 'price': 2500000}, {'City': 'Dubai', 'price': 6900000}, {'City': 'Dubai', 'price': 8749888}, {'City': 'Dubai', 'price': 1100000}, {'City': 'Ras Al Khaimah', 'price': 720000}, {'City': 'Dubai', 'price': 1200000}, {'City': 'Dubai', 'price': 3300000}, {'City': 'Dubai', 'price': 35000000}, {'City': 'Dubai', 'price': 6800000}, {'City': 'Dubai', 'price': 2000000}, {'City': 'Dubai', 'price': 910000}, {'City': 'Dubai', 'price': 1800000}, {'City': 'Dubai', 'price': 56035350}, {'City': 'Dubai', 'price': 1150000}, {'City': 'Dubai', 'price': 2750000}, {'City': 'Dubai', 'price': 3750000}, {'City': 'Dubai', 'price': 5000000}, {'City': 'Dubai', 'price': 2500000}, {'City': 'Ras Al Khaimah', 'price': 1300000}, {'City': 'Dubai', 'price': 520000}, {'City': 'Dubai', 'price': 3500000}, {'City': 'Dubai', 'price': 950000}, {'City': 'Dubai', 'price': 4250000}, {'City': 'Dubai', 'price': 750000}, {'City': 'Dubai', 'price': 10000000}, {'City': 'Dubai', 'price': 1700000}, {'City': 'Sharjah', 'price': 2089000}, {'City': 'Dubai', 'price': 2700000}, {'City': 'Dubai', 'price': 2200000}, {'City': 'Dubai', 'price': 3300000}, {'City': 'Dubai', 'price': 6850000}, {'City': 'Dubai', 'price': 9200000}, {'City': 'Dubai', 'price': 8870000}, {'City': 'Dubai', 'price': 8186200}, {'City': 'Dubai', 'price': 9000000}, {'City': 'Dubai', 'price': 20000000}, {'City': 'Dubai', 'price': 1599999}, {'City': 'Dubai', 'price': 1599999}, {'City': 'Dubai', 'price': 950000}, {'City': 'Dubai', 'price': 14000000}, {'City': 'Sharjah', 'price': 1999000}, {'City': 'Sharjah', 'price': 2399000}, {'City': 'Sharjah', 'price': 4899000}, {'City': 'Sharjah', 'price': 5199000}, {'City': 'Dubai', 'price': 3200000}, {'City': 'Dubai', 'price': 1100000}, {'City': 'Dubai', 'price': 22000000}, {'City': 'Dubai', 'price': 2995000}, {'City': 'Dubai', 'price': 2400000}, {'City': 'Dubai', 'price': 1200000}, {'City': 'Dubai', 'price': 11900000}, {'City': 'Dubai', 'price': 3000000}, {'City': 'Dubai', 'price': 2676000}, {'City': 'Dubai', 'price': 6700000}, {'City': 'Sharjah', 'price': 1100000}, {'City': 'Dubai', 'price': 6500000}, {'City': 'Dubai', 'price': 1300000}, {'City': 'Dubai', 'price': 5200000}, {'City': 'Abu Dhabi', 'price': 10500000}, {'City': 'Abu Dhabi', 'price': 949000}, {'City': 'Abu Dhabi', 'price': 9300000}, {'City': 'Dubai', 'price': 3900000}, {'City': 'Dubai', 'price': 1499000}, {'City': 'Dubai', 'price': 9000000}, {'City': 'Dubai', 'price': 1500000}, {'City': 'Dubai', 'price': 2700000}, {'City': 'Dubai', 'price': 24995000}, {'City': 'Dubai', 'price': 3300000}, {'City': 'Dubai', 'price': 11500000}, {'City': 'Ras Al Khaimah', 'price': 30000000}, {'City': 'Dubai', 'price': 3700000}, {'City': 'Dubai', 'price': 1475000}, {'City': 'Dubai', 'price': 2150000}, {'City': 'Dubai', 'price': 975000}, {'City': 'Dubai', 'price': 830000}, {'City': 'Dubai', 'price': 1590000}, {'City': 'Abu Dhabi', 'price': 8700000}, {'City': 'Dubai', 'price': 2250000}, {'City': 'Dubai', 'price': 3700000}, {'City': 'Dubai', 'price': 5600000}, {'City': 'Dubai', 'price': 5000000}, {'City': 'Abu Dhabi', 'price': 1180000}, {'City': 'Dubai', 'price': 1190000}, {'City': 'Dubai', 'price': 17000000}, {'City': 'Dubai', 'price': 950000}, {'City': 'Dubai', 'price': 2300000}, {'City': 'Abu Dhabi', 'price': 7300000}, {'City': 'Dubai', 'price': 11000000}, {'City': 'Dubai', 'price': 850000}, {'City': 'Dubai', 'price': 3100000}, {'City': 'Dubai', 'price': 1000000}, {'City': 'Dubai', 'price': 8186200}, {'City': 'Dubai', 'price': 20000000}, {'City': 'Dubai', 'price': 3200000}, {'City': 'Dubai', 'price': 3899999}, {'City': 'Dubai', 'price': 2850000}, {'City': 'Dubai', 'price': 2190000}, {'City': 'Abu Dhabi', 'price': 5999990}, {'City': 'Dubai', 'price': 4800000}, {'City': 'Dubai', 'price': 3800000}, {'City': 'Dubai', 'price': 18300000}, {'City': 'Dubai', 'price': 16000000}, {'City': 'Dubai', 'price': 7199999}, {'City': 'Sharjah', 'price': 4899000}, {'City': 'Sharjah', 'price': 1899000}, {'City': 'Sharjah', 'price': 2169000}, {'City': 'Sharjah', 'price': 6999000}, {'City': 'Dubai', 'price': 13000000}, {'City': 'Sharjah', 'price': 2399000}, {'City': 'Sharjah', 'price': 5999000}, {'City': 'Dubai', 'price': 12500000}, {'City': 'Dubai', 'price': 2200000}, {'City': 'Dubai', 'price': 19500000}, {'City': 'Dubai', 'price': 5400000}, {'City': 'Dubai', 'price': 2100000}, {'City': 'Dubai', 'price': 2000000}, {'City': 'Dubai', 'price': 540000}, {'City': 'Dubai', 'price': 3750000}, {'City': 'Dubai', 'price': 1350000}, {'City': 'Dubai', 'price': 11500000}, {'City': 'Abu Dhabi', 'price': 6300000}, {'City': 'Dubai', 'price': 6850000}, {'City': 'Dubai', 'price': 11200000}, {'City': 'Dubai', 'price': 4700000}, {'City': 'Dubai', 'price': 65000000}, {'City': 'Dubai', 'price': 800000}, {'City': 'Dubai', 'price': 2070800}, {'City': 'Dubai', 'price': 2900000}, {'City': 'Dubai', 'price': 2399000}, {'City': 'Dubai', 'price': 20000000}, {'City': 'Dubai', 'price': 75363600}, {'City': 'Sharjah', 'price': 769000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Ajman', 'price': 550000}, {'City': 'Abu Dhabi', 'price': 2950000}, {'City': 'Dubai', 'price': 2300000}, {'City': 'Dubai', 'price': 26800000}, {'City': 'Dubai', 'price': 9300000}, {'City': 'Dubai', 'price': 3850000}, {'City': 'Dubai', 'price': 11500000}, {'City': 'Abu Dhabi', 'price': 35000000}, {'City': 'Ajman', 'price': 535000}, {'City': 'Dubai', 'price': 8750000}, {'City': 'Dubai', 'price': 12300000}, {'City': 'Dubai', 'price': 2350000}, {'City': 'Dubai', 'price': 2200000}, {'City': 'Dubai', 'price': 7000000}, {'City': 'Sharjah', 'price': 1999000}, {'City': 'Sharjah', 'price': 4899000}, {'City': 'Sharjah', 'price': 5999000}, {'City': 'Dubai', 'price': 7000000}, {'City': 'Dubai', 'price': 5700000}, {'City': 'Dubai', 'price': 5200000}, {'City': 'Dubai', 'price': 2800000}, {'City': 'Dubai', 'price': 1250000}, {'City': 'Dubai', 'price': 1200000}, {'City': 'Ras Al Khaimah', 'price': 350000}, {'City': 'Dubai', 'price': 7500000}, {'City': 'Dubai', 'price': 1800000}, {'City': 'Dubai', 'price': 1050000}, {'City': 'Dubai', 'price': 2400000}, {'City': 'Dubai', 'price': 3700000}, {'City': 'Abu Dhabi', 'price': 990000}, {'City': 'Dubai', 'price': 710000}, {'City': 'Dubai', 'price': 1450000}, {'City': 'Ajman', 'price': 299999}, {'City': 'Dubai', 'price': 1650000}, {'City': 'Sharjah', 'price': 829000}, {'City': 'Dubai', 'price': 870000}, {'City': 'Dubai', 'price': 3900000}, {'City': 'Dubai', 'price': 4700000}, {'City': 'Dubai', 'price': 2850000}, {'City': 'Dubai', 'price': 725000}, {'City': 'Dubai', 'price': 1700000}, {'City': 'Dubai', 'price': 3200000}, {'City': 'Dubai', 'price': 40000000}, {'City': 'Dubai', 'price': 2250000}, {'City': 'Dubai', 'price': 199000000}, {'City': 'Dubai', 'price': 3499000}, {'City': 'Dubai', 'price': 3190000}, {'City': 'Dubai', 'price': 5800000}, {'City': 'Dubai', 'price': 3750000}, {'City': 'Dubai', 'price': 2350000}, {'City': 'Dubai', 'price': 754993}, {'City': 'Dubai', 'price': 1530000}, {'City': 'Dubai', 'price': 25000000}, {'City': 'Dubai', 'price': 3900000}, {'City': 'Dubai', 'price': 7700000}, {'City': 'Sharjah', 'price': 2049000}, {'City': 'Dubai', 'price': 10200000}, {'City': 'Dubai', 'price': 3850000}, {'City': 'Dubai', 'price': 2300000}, {'City': 'Dubai', 'price': 634999}, {'City': 'Ras Al Khaimah', 'price': 1375000}, {'City': 'Ras Al Khaimah', 'price': 2499000}, {'City': 'Ras Al Khaimah', 'price': 380000}, {'City': 'Dubai', 'price': 1350000}, {'City': 'Dubai', 'price': 2850000}, {'City': 'Abu Dhabi', 'price': 1499999}, {'City': 'Dubai', 'price': 3300000}, {'City': 'Dubai', 'price': 14500000}, {'City': 'Dubai', 'price': 1250000}, {'City': 'Dubai', 'price': 3450000}, {'City': 'Dubai', 'price': 6000000}, {'City': 'Dubai', 'price': 2100000}, {'City': 'Sharjah', 'price': 768000}, {'City': 'Sharjah', 'price': 1298990}, {'City': 'Dubai', 'price': 6850000}, {'City': 'Dubai', 'price': 1690000}, {'City': 'Dubai', 'price': 20000000}, {'City': 'Abu Dhabi', 'price': 4600000}, {'City': 'Dubai', 'price': 3650000}, {'City': 'Dubai', 'price': 4500000}, {'City': 'Sharjah', 'price': 2169900}, {'City': 'Dubai', 'price': 900000}, {'City': 'Dubai', 'price': 7150000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Dubai', 'price': 1800000}, {'City': 'Dubai', 'price': 1438000}, {'City': 'Dubai', 'price': 3785000}, {'City': 'Dubai', 'price': 15000000}, {'City': 'Ajman', 'price': 311000}, {'City': 'Dubai', 'price': 2111111}, {'City': 'Abu Dhabi', 'price': 7200000}, {'City': 'Dubai', 'price': 2847837}, {'City': 'Dubai', 'price': 5800000}, {'City': 'Ajman', 'price': 300000}, {'City': 'Ras Al Khaimah', 'price': 1500000}, {'City': 'Ras Al Khaimah', 'price': 6500000}, {'City': 'Dubai', 'price': 559000}, {'City': 'Sharjah', 'price': 2049000}, {'City': 'Dubai', 'price': 5999000}, {'City': 'Sharjah', 'price': 2089000}, {'City': 'Ras Al Khaimah', 'price': 700000}, {'City': 'Sharjah', 'price': 2399000}, {'City': 'Sharjah', 'price': 5999000}, {'City': 'Ajman', 'price': 519999}, {'City': 'Dubai', 'price': 3499999}, {'City': 'Abu Dhabi', 'price': 2600000}, {'City': 'Dubai', 'price': 755000}, {'City': 'Dubai', 'price': 1190000}, {'City': 'Dubai', 'price': 1323000}, {'City': 'Dubai', 'price': 6900000}, {'City': 'Dubai', 'price': 3100000}, {'City': 'Sharjah', 'price': 5190000}, {'City': 'Dubai', 'price': 1300000}, {'City': 'Dubai', 'price': 2700000}, {'City': 'Dubai', 'price': 5099888}, {'City': 'Sharjah', 'price': 1280000}, {'City': 'Dubai', 'price': 10000000}, {'City': 'Sharjah', 'price': 1745000}, {'City': 'Dubai', 'price': 32000000}, {'City': 'Dubai', 'price': 3300000}, {'City': 'Dubai', 'price': 2341775}, {'City': 'Abu Dhabi', 'price': 6000000}, {'City': 'Dubai', 'price': 1300000}, {'City': 'Dubai', 'price': 800000}, {'City': 'Sharjah', 'price': 2089000}, {'City': 'Abu Dhabi', 'price': 8500000}, {'City': 'Sharjah', 'price': 2049000}, {'City': 'Dubai', 'price': 3350000}, {'City': 'Dubai', 'price': 5800000}, {'City': 'Dubai', 'price': 479000}, {'City': 'Dubai', 'price': 4400000}, {'City': 'Sharjah', 'price': 5199000}, {'City': 'Dubai', 'price': 1600000}, {'City': 'Dubai', 'price': 3243160}, {'City': 'Ajman', 'price': 340000}, {'City': 'Dubai', 'price': 675000}, {'City': 'Dubai', 'price': 2450000}, {'City': 'Dubai', 'price': 1450000}, {'City': 'Dubai', 'price': 750000}, {'City': 'Dubai', 'price': 2400000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Dubai', 'price': 1100000}, {'City': 'Dubai', 'price': 13000000}, {'City': 'Sharjah', 'price': 4895000}, {'City': 'Sharjah', 'price': 5990000}, {'City': 'Dubai', 'price': 1899000}, {'City': 'Dubai', 'price': 999999}, {'City': 'Abu Dhabi', 'price': 20000000}, {'City': 'Dubai', 'price': 5130187}, {'City': 'Dubai', 'price': 2548080}, {'City': 'Sharjah', 'price': 5199000}, {'City': 'Dubai', 'price': 1700000}, {'City': 'Dubai', 'price': 60000000}, {'City': 'Dubai', 'price': 2550000}, {'City': 'Dubai', 'price': 750000}, {'City': 'Dubai', 'price': 1040000}, {'City': 'Sharjah', 'price': 1935000}, {'City': 'Sharjah', 'price': 2549000}, {'City': 'Sharjah', 'price': 2049000}, {'City': 'Sharjah', 'price': 5999000}, {'City': 'Sharjah', 'price': 1999000}, {'City': 'Sharjah', 'price': 2399000}, {'City': 'Dubai', 'price': 1190000}, {'City': 'Abu Dhabi', 'price': 19500000}, {'City': 'Abu Dhabi', 'price': 9500000}, {'City': 'Dubai', 'price': 8500000}, {'City': 'Ras Al Khaimah', 'price': 2675000}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Dubai', 'price': 4800000}, {'City': 'Dubai', 'price': 6300000}, {'City': 'Abu Dhabi', 'price': 4500000}, {'City': 'Abu Dhabi', 'price': 23000000}, {'City': 'Dubai', 'price': 1300000}, {'City': 'Dubai', 'price': 1400000}, {'City': 'Dubai', 'price': 5000000}, {'City': 'Abu Dhabi', 'price': 730000}, {'City': 'Sharjah', 'price': 2100000}, {'City': 'Abu Dhabi', 'price': 730000}, {'City': 'Dubai', 'price': 1600000}, {'City': 'Dubai', 'price': 11000000}, {'City': 'Dubai', 'price': 2502662}, {'City': 'Dubai', 'price': 4200000}, {'City': 'Sharjah', 'price': 749000}, {'City': 'Dubai', 'price': 5750000}, {'City': 'Dubai', 'price': 2650000}, {'City': 'Abu Dhabi', 'price': 5200000}, {'City': 'Ras Al Khaimah', 'price': 3007888}, {'City': 'Dubai', 'price': 4000000}])

In [38]:
data_cities

,City,price
0,Dubai,2500000
1,Dubai,7200000
2,Dubai,950000
3,Dubai,9999990
4,Dubai,2100000
...,...,...
558,Dubai,5750000
559,Dubai,2650000
560,Abu Dhabi,5200000
561,Ras Al Khaimah,3007888


In [26]:
df1 = pd.DataFrame(data_dict["result"])

In [27]:
df1

,addedOn_year_month,property_count
0,2023-09,1
1,2024-02,2
2,2024-03,2
3,2024-04,1
4,2024-05,10
5,2024-06,13
6,2024-07,46
7,2024-08,28


In [12]:
df1["latitude"].min()

25.3071588

In [8]:
query = 'Data Intent:\nThe final table should include properties that are located north to Dubai Mall and are in the 30th percentile in that group. The table should include the following columns: title, bathrooms, bedrooms, type, price, verified, priceDuration, sizeMin, furnishing, description, latitude, longitude, City, addedOn_year_month, addedOn_month_name, addedOn_month_num, addedOn_day_num, and Area. The rows should be grouped by the location (north to Dubai'

In [5]:
sample = pd.read_parquet("../data/sample_data.parquet")

In [10]:
sample.dtypes

title                  object
bathrooms               int64
bedrooms                int64
type                   object
price                   int64
verified                 bool
priceDuration          object
sizeMin                 int64
furnishing             object
description            object
latitude              float64
longitude             float64
City                   object
addedOn_year_month     object
addedOn_month_name     object
addedOn_month_num       int64
addedOn_day_num         int64
Area                   object
dtype: object

In [9]:
sample.to_dict(orient="records")

[{'title': 'Great ROI I High Floor I Creek View',
  'bathrooms': 3,
  'bedrooms': 2,
  'type': 'Residential for Sale',
  'price': 2500000,
  'verified': True,
  'priceDuration': 'sell',
  'sizeMin': 44,
  'furnishing': 'NO',
  'description': 'MNA Properties is delighted to offer this apartment located in the well-esteemed Binghatti Canal, situated in the heart of Business Bay, Dubai, UAE. The 2BR apartment with the most attractive layout based on the high floor and offers the breathtaking panoramic view of Business Bay Canal; near Burj Khalifa.\n\nTo arrange a viewing or to find out more, please contact Amir via phone or Whatsapp on +971581684981 or email at amir.m@mnaproperties.ae\n\nProperty Features:\n2 Bedrooms\n2 Bathrooms + Guest room\nSize 1,323 sq. ft.\n2 Balconies\nPrivate Jacuzzi / Small Pool\nCanal View\n\nCan be sold Unfurnished as well\n\nBinghatti Canal Residences is a newly built project in Business Bay, Dubai that features symbolic design elements to generate a sense of

In [ ]:
from src.agent import main_agent

In [ ]:
main_agent("show on map 2 bedroom flats that are cheaper than 2 million and in Dubai")

In [ ]:
USER_QUERY = "show on map 2 bedroom flats that are cheaper than 2 million and in Dubai"

command = extract_data_intent.invoke(USER_QUERY)

In [ ]:
command

In [ ]:
data_json_str = safe_dataframe_tool.invoke(command)
data_dict = json.loads(data_json_str)

In [ ]:
pd.DataFrame(data_dict["result"])['latitude']

In [ ]:
pd.DataFrame(data_dict["result"])

In [ ]:
data_json_str

In [ ]:
data_dict


In [ ]:
pd.DataFrame(data_dict["result"])

In [ ]:
def main_agent(query: str):
    # Step 1: Check if query is relevant
    if not is_uae_real_estate_query(query):
        return {"type": "output", "data": "This is an irrelevant question to UAE property."}

    # Step 2: Extract data intent from user query
    data_intent = extract_data_intent.invoke(query)

    # Step 3: Use PandasAI to retrieve relevant data
    data_json_str = safe_dataframe_tool.invoke(data_intent)
    data_dict = json.loads(data_json_str)

    if not data_dict.get("success"):
        return {"type": "error", "error": data_dict.get("error"), "solution": data_dict.get("solution")}

    # Step 4: Classify the user's goal
    action = llm_classifier(query)
    

    # Step 5: Take action based on user intent
    if action == "output":
        return {"type": "data", "data": data_dict["result"]}
    
    
    elif action == "plot_stats":

        viz_input = json.dumps({"data": data_dict["result"], "query": query})
        result_json_str = create_plotly_code.invoke(viz_input)
        result = json.loads(result_json_str)
        return {"type": "plot", **result}

    elif action == "geospatial_plot":
        
        html = generate_google_maps_html(data_dict["result"])
        return {"type": "html", "content": html}

    else:
        return {"type": "error", "error": f"Unknown action '{action}' from classifier."}


In [ ]:
result = main_agent("show on map 2 bedroom flats that are cheaper than 2 million and in Dubai")

In [ ]:
result

In [ ]:
data_dict

In [ ]:
from src.tools import get

In [ ]:
DATA_DIR = r"../data/uae_real_estate_2024_geo_ready2.parquet"
df =  pd.read_parquet(DATA_DIR)

In [ ]:
df.drop(columns="addedOn", inplace=True)

In [ ]:
df.columns

In [ ]:
df.columns[df.isna().any()].tolist()


In [ ]:
df['bathrooms'].fillna(0, inplace=True)
df['bedrooms'].fillna(0, inplace=True)
df['furnishing'] = df['furnishing'].cat.add_categories('Unknown')
df['furnishing'] = df['furnishing'].fillna('Unknown')


In [ ]:
df["sizeMin"] = df["bedrooms"].apply(lambda x: 20 + 12 * x)

In [ ]:
df['addedOn'] = df['addedOn'].dt.strftime('%Y-%m-%dT%H:%M:%S')

In [ ]:
df["displayAddress"]

In [ ]:
df.columns[df.isna().any()].tolist()


In [ ]:
df.to_csv(r"../data/uae_real_estate_2024_geo_ready2.csv", index=False)

In [ ]:
import pandasai as pai

companies = pai.create(
  path="my-org/properties3",
  df=pai.read_csv("../data/uae_real_estate_2024_geo_ready2.csv"),
  description="UAE properties dataset 2"
)

In [ ]:
result = companies.chat("Retrieve properties located within a 2-mile radius of Dubai Eye.")